In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

pd.set_option('display.max_columns', 1000)

In [2]:
dataset = pd.read_pickle('other_xmltodict_all_major_fields.pkl')

In [3]:
### Drop the rows with no values in section_bodies
dataset = dataset[dataset['section_bodies'] != 'Null']
print("Without Null rows: " + str(len(dataset)))

Without Null rows: 2520


In [4]:
#only keep the SOR and CRC...SI are not included in the scope
dataset['regulation_type'].value_counts()

SOR     2265
SI       246
Null       9
Name: regulation_type, dtype: int64

In [5]:
dataset = dataset[dataset['regulation_type']!= 'SI']

## 1. Creating the Corpus & Sparse Matrix

In [6]:
def make_corpus(dataset, column):
    '''This function reads in a pandas dataframe (dataset) and the column name (e.g. 'text_column'). 
    Then it strips all punctuation, makes it lower case, tokenizes it, and removes english stopwords. 
    Finally it returns two lists (either directly or via tuple unpacking. the first is the documents
    without stopwords; the second is the documents stemmed via porterstemmer.
    '''
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    stop_words = set(stopwords.words('english'))
    import string 
    import sys
    translator = str.maketrans('', '', string.punctuation)
    import re
    from nltk.stem import PorterStemmer
    ps = PorterStemmer()
    filtered_text_list = []
    stemmed_text_list = []
    for i in range (0, len(dataset)):
        text = dataset.iloc[i][column].lower().translate(translator)
        text = re.sub('\n', ' ', text)
        word_tokens = word_tokenize(text) 
        filtered_text = [w for w in word_tokens if not w in stop_words]
        filtered_text = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_text.append(w)
        stemmed_text = []
        for w in filtered_text:
            stemmed_text.append(ps.stem(w))
        filtered_text_list.append(str(filtered_text).translate(translator))
        stemmed_text_list.append(str(stemmed_text).translate(translator))
        sys.stdout.write("\r" + "Creating Corpus.. Processing Record: " + str(i+1) + " of " + str(len(dataset)))
        sys.stdout.flush()
    print("\nDONE!")
    return filtered_text_list, stemmed_text_list

In [7]:
filtered_text, stemmed_text = make_corpus(dataset,'section_bodies')

Creating Corpus.. Processing Record: 2274 of 2274
DONE!


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(stemmed_text).toarray()

In [9]:
X.shape

(2274, 12595)

In [13]:
df_sparse = pd.DataFrame(X, columns = cv.get_feature_names())

## 2. Cleaning the Sparse Matrix

In [14]:
features = pd.Series(df_sparse.columns)

##### Removing features which contain a number

In [15]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [16]:
#remove all the features which are numeric
index = features.map(hasNumbers)

In [17]:
#how many are numeric?
index.value_counts()

False    7512
True     5083
dtype: int64

In [18]:
#list of features to drop
dropped_features = features[index]

In [19]:
len(dropped_features)

5083

In [20]:
#Drop it like its hot
df_sparse = df_sparse.drop(columns=dropped_features)

In [21]:
df_sparse.shape

(2274, 7512)

##### Removing the helping verbs

In [22]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [23]:
#There are 23 helping verbs in total
helping_verbs = ['am', 'is', 'are', 'was', 'were', 'being', 'been', 'be', 'have', 'has', 'had', 'do', 'does', 
                 'did', 'will', 'would', 'shall', 'should', 'may', 'might', 'must', 'can', 'could']

#Obtain the stemed version of each word
helping_verbs_stemmed = []
for verb in helping_verbs:
    helping_verbs_stemmed.append(ps.stem(verb).lower())

In [25]:
#Which helping verbs are in the sparse matrix?
helping_verbs_stemmed_drop = []

cols = list(df_sparse.columns)

for verb in helping_verbs_stemmed:
    if verb in cols:
        helping_verbs_stemmed_drop.append(verb)
        
print (helping_verbs_stemmed_drop)

['will', 'would', 'shall', 'may', 'might', 'must', 'could']


In [28]:
#Remove the helping verbs and check the outcomes
print ('shape before dropping helping verbs:' + str(df_sparse.shape))

df_sparse = df_sparse.drop(columns=helping_verbs_stemmed_drop)

print ('shape after dropping helping verbs:' + str(df_sparse.shape))

shape before dropping helping verbs:(2274, 7512)
shape after dropping helping verbs:(2274, 7505)


## 3. TF-IDF Vectorizer

In [29]:
#convert the count matrix to tf-idf representation
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(df_sparse).toarray()

In [141]:
#df_sparse = pd.DataFrame(tfidf, columns = cols)

In [30]:
feature_names = df_sparse.columns

# find maximum value for each of the features over all of dataset:
max_val = df_sparse.max(axis=0).ravel()

#sort weights from smallest to biggest and extract their indices 
sort_by_tfidf = max_val.argsort()

print("Features with lowest tfidf:\n{}".format(
      feature_names[sort_by_tfidf[:20]]))

print("\nFeatures with highest tfidf: \n{}".format(
      feature_names[sort_by_tfidf[-20:]]))

Features with lowest tfidf:
Index(['northwestel', 'speech', 'premiumpay', 'spednic', 'spectat',
       'discredit', 'prepaidport', 'discriminatori', 'sphere', 'iran', 'ipc',
       'ioniz', 'ionexchang', 'dish', 'dishonest', 'sovereignti', 'ioc',
       'sparkler', 'discomfort', 'discolor'],
      dtype='object')

Features with highest tfidf: 
Index(['appeal', 'particular', 'system', 'categori', 'licenc', 'produc',
       'hazard', 'act', 'particip', 'certif', 'ferri', 'store', 'vessel',
       'subsect', 'dive', 'suppli', 'invest', 'food', 'plan', 'explos'],
      dtype='object')


## 4. Latent Semantic Analysis

In [32]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components = 100, algorithm='arpack', random_state=0)
dtm_lsa = lsa.fit_transform(tfidf)

In [33]:
#What are the top components of each concept?
for i, comp in enumerate(lsa.components_):
    terms_in_comp = zip(cols,comp)
    sorted_terms = sorted(terms_in_comp, key=lambda x: x[1], reverse=True)[:10]
    print('Concept %d:' %i)
    
    for term in sorted_terms:
        print(term[0])
    print (' ')
    

Concept 0:
forbid
come
datum
regi
registrar
lamproom
optim
nwtsoirc
secretari
perpendicular
 
Concept 1:
lamproom
nwtsoirc
plywood
element
airport
build
stripe
appli
grown
oviduct
 
Concept 2:
board
act
perpendicular
subrul
marker
commod
author
poult
secretari
purg
 
Concept 3:
board
commod
marker
poult
exempt
protrud
interpret
tp
optim
exponentialtimeaverag
 
Concept 4:
immov
prisonerofwar
articl
convent
remarri
canada
ordinari
optim
extens
grandpar
 
Concept 5:
remarri
grandpar
gone
implic
condit
optim
custodian
pave
paddleboat
dustproof
 
Concept 6:
compani
bank
entireti
remarri
hoist
grandpar
insul
invert
acquir
subsidiari
 
Concept 7:
gone
station
canada
implic
registrar
chapter
exponentialtimeaverag
come
forbid
enabl
 
Concept 8:
permafrost
exponentialtimeaverag
gone
author
regi
implic
optim
compani
scare
bank
 
Concept 9:
insul
proclaim
miner
loadcarri
lever
applic
board
lend
compani
prioriti
 
Concept 10:
column
servant
scare
proclaim
lever
italic
amount
paus
font
pave
 
Concep

Concept 95:
vat
firstclass
crossventil
enabl
placement
function
resolv
act
lever
tankless
 
Concept 96:
park
shaft
tundra
sheriff
shall
psig
adapt
notic
scoop
ignitionprotect
 
Concept 97:
featur
amend
class
offenc
benefit
desert
draw
scoop
notic
certif
 
Concept 98:
account
corpor
advanc
april
propan
sheriff
offenc
paddleboat
protrud
tire
 
Concept 99:
program
class
assess
featur
agenc
eldorado
regim
hm
contributor
activ
 


In [35]:
list_concepts = []
    
for i in range(0,len(lsa.components_)):
    list_concepts.append('Concept %d' %i)
    
df_lsa = pd.DataFrame(dtm_lsa, columns=list_concepts)

df_lsa

,Concept 0,Concept 1,Concept 2,Concept 3,Concept 4,Concept 5,Concept 6,Concept 7,Concept 8,Concept 9,Concept 10,Concept 11,Concept 12,Concept 13,Concept 14,Concept 15,Concept 16,Concept 17,Concept 18,Concept 19,Concept 20,Concept 21,Concept 22,Concept 23,Concept 24,Concept 25,Concept 26,Concept 27,Concept 28,Concept 29,Concept 30,Concept 31,Concept 32,Concept 33,Concept 34,Concept 35,Concept 36,Concept 37,Concept 38,Concept 39,Concept 40,Concept 41,Concept 42,Concept 43,Concept 44,Concept 45,Concept 46,Concept 47,Concept 48,Concept 49,Concept 50,Concept 51,Concept 52,Concept 53,Concept 54,Concept 55,Concept 56,Concept 57,Concept 58,Concept 59,Concept 60,Concept 61,Concept 62,Concept 63,Concept 64,Concept 65,Concept 66,Concept 67,Concept 68,Concept 69,Concept 70,Concept 71,Concept 72,Concept 73,Concept 74,Concept 75,Concept 76,Concept 77,Concept 78,Concept 79,Concept 80,Concept 81,Concept 82,Concept 83,Concept 84,Concept 85,Concept 86,Concept 87,Concept 88,Concept 89,Concept 90,Concept 91,Concept 92,Concept 93,Concept 94,Concept 95,Concept 96,Concept 97,Concept 98,Concept 99
0,0.133869,0.009151,0.163808,-0.116088,0.016744,0.011887,-0.094306,-0.011585,0.037939,-0.175331,0.345300,0.151291,-0.149515,-0.043991,0.011330,0.046791,0.063265,-0.100130,0.070397,0.131186,-0.072668,-0.110688,-0.072583,0.043131,0.038753,0.002544,0.274045,-0.127520,0.099704,-0.032315,-0.063223,0.309143,0.136656,-0.009330,0.032651,-0.079209,0.078531,0.020441,-0.059283,-0.155243,-0.024651,0.125597,-0.050829,0.102534,-0.008854,0.054635,0.039810,-0.029672,0.093739,-0.084810,0.026407,-0.097363,0.053123,-0.025429,-0.029235,0.063678,0.037002,3.623601e-03,-0.013142,0.096728,-0.028526,0.068509,0.073763,0.035280,-0.023833,-0.027624,0.030543,-0.012879,0.003534,-0.010372,0.008721,0.031634,0.058234,0.008837,-0.020180,0.041570,-0.048314,0.019509,-0.037781,0.081859,0.018719,-0.050301,0.039456,-0.063776,-0.015486,0.005744,-0.003512,0.023557,-0.032751,-0.015980,0.016520,-0.048725,0.010458,-0.007559,-0.024895,-0.029319,0.008756,-0.019034,0.002844,0.016143
1,0.040606,0.006009,0.050907,-0.036570,0.023986,0.006046,-0.016135,0.031183,-0.006259,0.007440,0.059679,0.014844,-0.005533,0.001092,0.001951,0.015375,0.006001,-0.015494,0.043211,-0.017797,-0.005461,-0.034303,-0.016443,0.032498,0.009691,-0.012396,0.013555,0.027627,0.015275,0.027867,-0.043693,-0.030031,-0.001740,0.020797,0.010958,-0.012952,0.007577,-0.003907,-0.000254,-0.007110,0.003372,-0.002505,0.008646,0.029338,0.017723,-0.032044,0.036203,0.037802,0.013838,0.048920,0.032897,0.020530,-0.003634,0.021591,0.008657,-0.034911,-0.030603,-2.291098e-02,0.002577,-0.105327,-0.004768,-0.064606,0.009019,0.046242,0.038407,0.032114,0.028951,-0.013300,0.005073,0.002715,-0.030343,-0.022097,-0.033708,0.050182,-0.027979,-0.040575,0.071214,-0.041731,0.059895,0.005634,0.021192,0.022351,0.017944,0.071310,0.002616,0.021564,0.000063,-0.032226,-0.000331,-0.002474,0.040784,0.023176,0.004360,0.007491,0.083713,0.050721,-0.071069,-0.053899,-0.040355,0.032900
2,0.431893,0.737588,-0.145732,0.047491,0.031886,0.017033,0.013733,-0.013343,0.042420,-0.006422,0.014444,0.018418,0.007394,0.022134,0.006597,0.003155,-0.022483,0.030875,-0.001983,0.046125,-0.020644,-0.036410,-0.022121,-0.047174,0.007922,0.084058,0.003500,0.032514,-0.029697,0.029179,-0.000315,-0.003458,0.011764,-0.002432,0.003308,-0.003573,-0.017009,0.006628,-0.017504,-0.021680,-0.002770,0.009269,-0.004329,0.024108,-0.046424,0.009715,0.003663,0.038887,-0.033378,0.011762,-0.026898,-0.015461,0.002626,-0.015188,-0.030096,-0.016173,-0.006494,2.797625e-02,-0.048303,0.006331,-0.027902,0.010210,-0.005972,-0.036171,0.022431,-0.021606,0.021355,0.006078,0.030416,-0.022505,0.025521,-0.029698,-0.011564,0.005830,0.006485,0.003723,-0.011735,0.042538,0.015574,0.009830,-0.012730,0.051077,0.017495,0.004754,-0.036551,0.011860,0.029957,-0.003741,0.001877,-0.012793,0.012078,0.000971,-0.005438,-0.002583,-0.011776,-0.007992,-0.018537,-0.025496,-0.005225,-0.005733
3,0.075667,-0.005102,0.106623,-0.051184,0.139599,0.183566,0

###### Document Similarity using LSA

In [37]:
from sklearn.preprocessing import Normalizer
df_norm = Normalizer(copy=False).fit_transform(dtm_lsa)

In [38]:
similarity = np.asarray(np.asmatrix(df_norm) * np.asmatrix(df_norm).T)
regs = list(dataset['instrument_number'])

In [39]:
df_heatmap = pd.DataFrame(similarity,index=regs, columns=regs)

In [40]:
df_heatmap

SOR/2017-109  SOR/85-567  SOR/93-293  SOR/98-87  \
SOR/2017-109         1.000000    0.037609    0.065081   0.091289   
SOR/85-567           0.037609    1.000000    0.043468   0.068407   
SOR/93-293           0.065081    0.043468    1.000000   0.053443   
SOR/98-87            0.091289    0.068407    0.053443   1.000000   
C.R.C., c. 208       0.045415    0.029254    0.055368   0.141263   
SOR/86-1135          0.018775    0.033047    0.923411   0.001302   
C.R.C., c. 220       0.030357    0.010036    0.049131   0.011986   
SOR/2007-121         0.053113    0.153126    0.051752  -0.001179   
SOR/2017-121         0.122646    0.068078    0.074645   0.121855   
SOR/57-216          -0.012544   -0.022523    0.027283   0.047852   
SOR/93-522           0.021964    0.053128    0.966448   0.038048   
SOR/91-74            0.195009    0.022727    0.042435   0.233201   
SOR/98-50            0.054836   -0.028207    0.033187   0.133397   
SOR/98-44            0.115418    0.141121    0.038400   0.130916   
C.R.C., c. 1359      0.091463    0.070774    0.098303   0.063989   
SOR/2011-91          0.037768    0.030438    0.067705   0.003508   
SOR/82-719           0.026759    0.009615    0.067373   0.020857   
SOR/2005-232         0.088771    0.104061    0.036279  -0.005361   
SOR/2011-170         0.064550    0.097999    0.044758   0.051839   
C.R.C., c. 591       0.103074    0.083890    0.051377  -0.038415   
SOR/94-562           0.032918    0.083691    0.017246   0.031159   
SOR/92-296           0.006880    0.050889    0.078410   0.014823   
C.R.C., c. 778       0.002248    0.089964   -0.005352   0.458288   
SOR/82-296           0.017204    0.038625    0.904601  -0.007601   
SOR/2001-399         0.079901    0.125887    0.050045   0.013702   
SOR/2012-17          0.037059    0.007137    0.074534   0.121185   
SOR/92-282          -0.002595   -0.018293    0.022720   0.016988   
SOR/2008-67          0.050799    0.027396    0.019140   0.018157   
SOR/2015-36          0.041888    0.009064    0.066025   0.190373   
SOR/2016-124         0.062274    0.016661    0.126246   0.051399   
...                       ...         ...         ...        ...   
SOR/2015-13          0.036161   -0.016144    0.022774   0.011094   
C.R.C., c. 88        0.086739    0.046507    0.881977   0.042997   
C.R.C., c. 1195     -0.001850    0.017633    0.115158  -0.017360   
SOR/2000-187         0.782929    0.107780    0.056838   0.024679   
SOR/2001-394         0.035637    0.097540    0.024404   0.076932   
SOR/2001-66          0.031315    0.035387    0.060659  -0.087637   
SOR/2006-115         0.035692    0.007125    0.024609   0.012302   
SOR/2008-208         0.019610   -0.006102    0.063856   0.095720   
SOR/91-45            0.033612    0.011076    0.057712   0.032367   
SOR/83-507           0.091437    0.011801    0.055702  -0.028377   
SOR/98-61            0.084029    0.213281    0.091877   0.129072   
SOR/2016-81          0.035692    0.007125    0.024609   0.012302   
SOR/89-348a          0.061061    0.041147    0.032578   0.137472   
SOR/84-962           0.149202   -0.012437    0.298595   0.236701   
SOR/98-49            0.088887   -0.015672    0.037789   0.202502   
SOR/2015-203         0.029541    0.005551    0.031405   0.024782   
C.R.C., c. 1354      0.147293    0.085785    0.088130   0.173652   
SOR/91-86            0.019719   -0.022861    0.027864   0.030379   
SOR/90-375           0.015310    0.058394    0.014561   0.159491   
SOR/98-213           0.032898    0.031990    0.035844   0.024420   
SOR/98-207           0.070850    0.033642    0.024841   0.004443   
C.R.C., c. 239       0.119218    0.030828    0.060084  -0.006946   
SOR/88-575           0.076382    0.063649    0.023224   0.052367   
SOR/91-172           0.019630    0.031563    0.928440   0.005434   
SOR/2005-16          0.122613    0.007822    0.032643   0.019967   
SOR/2016-317         0.046787    0.033132    0.021972  -0.023892   
SOR/2018-47          0.067156   -0.003602    0.020266   0.089224   
SOR/2

In [41]:
'''
After color coding in excel, it can be easily seen which regulatiosn are very similar to eachother. Upon manually
opening a couple, it can be seen that often regulations will attempt to convey similar (if not identical) meanings
with inconsistent language
'''

#export to excel for analysis...seaborn heatmap takes too long to load on my machine
df_heatmap.to_excel('LSA_Doc_Similarity_v1.xlsx')

## 5. Extracting the highly similar documents

In [126]:
#minimum score/value for regulations to be considered similar
thres_min = 0.95

#max score/value, above which regulations are the same document
thres_max = 0.999999

In [141]:
#column names 
regs = list(df_heatmap.columns)

#master list of similar regs to append to
similar_regs = []

#analyze each column's scores one at a time
for reg_1 in regs:
    reg_1_scores = list(df_heatmap[reg_1])
    
    #track the index value (integer) which meet the criteria
    index_value = 0
    list_index = []

    for score in reg_1_scores:
        if score > thres_min and score < thres_max:
            list_index.append(index_value)
        index_value += 1
    
    #track the reg pairs which exceed the criteria - check for duplicates
    for i in list_index:
        reg_2 = df_heatmap.index[i]
        
        if [reg_2,reg_1] not in similar_regs:
            similar_regs.append([reg_1,reg_2])
    

## Which Lebels in the Regs are Most Similar?